In [37]:
import numpy as np
import re
import nltk
from sklearn.datasets import  load_files
nltk.download('stopwords')
nltk.download('wordnet')
import  pickle
from nltk.corpus import stopwords
import exchelp.exception_helper as eh
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/olgunerguzel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/olgunerguzel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [39]:
## load data

try:
    movie_data = load_files(r"data/raw/bbc4")
except Exception as e:
    eh.CoreException('data load failed',e,logIt=True,dontThrow=True,shouldExit=True).Act()
    
X, y = movie_data.data, movie_data.target

from sklearn.model_selection import train_test_split
X_trainData, X_testData, y_trainData, y_testData = train_test_split(X, y, test_size=0.2, random_state=0)


In [40]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    documents.append(document)

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
countVectorized = vectorizer.fit_transform(documents).toarray()

from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
tfidTransformed = tfidfconverter.fit_transform(countVectorized).toarray()

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidTransformed, y, test_size=0.2, random_state=0)

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=0)

In [43]:
y_pred = classifier.predict(X_test)


from sklearn.metrics import classification_report
target_names = ['business', 'entertainment', 'politics','sport','tech']
print(classification_report(y_test, y_pred, target_names=target_names))

print(X_testData[10])
print(y_pred[10])

print("-------------------------------------------")
print(X_testData[15])
print(y_pred[15])

               precision    recall  f1-score   support

     business       0.95      0.94      0.95       102
entertainment       0.99      0.94      0.96        84
     politics       0.95      0.95      0.95        78
        sport       0.97      1.00      0.99       103
         tech       0.95      0.97      0.96        78

     accuracy                           0.96       445
    macro avg       0.96      0.96      0.96       445
 weighted avg       0.96      0.96      0.96       445

b'Dogged Federer claims Dubai crown\n\nWorld number one Roger Federer added the Dubai Championship trophy to his long list of successes - but not before he was given a test by Ivan Ljubicic.\n\nTop seed Federer looked to be on course for a easy victory when he thumped the eighth seed 6-1 in the first set. But Ljubicic, who beat Tim Henman in the last eight, dug deep to secure the second set after a tense tiebreak. Swiss star Federer was not about to lose his cool, though, turning on the style to w